In [1]:
import locale
import pandas as pd

# Import Rangekeeper:
import rangekeeper as rk

In [2]:
locale.setlocale(locale.LC_ALL, '')
units = rk.measure.Index.registry
currency = rk.measure.register_currency(registry=units)
print(currency)

Rangekeeper Measure: "Australian Dollar". Currency of ['AUSTRALIA', 'COCOS (KEELING) ISLANDS', 'CHRISTMAS ISLAND', 'HEARD & MCDONALD ISLANDS', 'KIRIBATI', 'NORFOLK ISLAND', 'NAURU', 'TUVALU']. Units: AUD


In [3]:
transactions = {
    pd.Timestamp('2020-01-01'): 100,
    pd.Timestamp('2020-01-02'): 200,
    pd.Timestamp('2019-01-01'): 300,
    pd.Timestamp('2020-12-31'): -100
    }

movements = pd.Series(data=transactions)

cash_flow = rk.flux.Flow(
    name='Operational Expenses',
    movements=movements,
    units=currency.units)

In [4]:
cash_flow

date,Operational Expenses
2020-01-01 00:00:00,$100.00
2020-01-02 00:00:00,$200.00
2019-01-01 00:00:00,$300.00
2020-12-31 00:00:00,-$100.00


In [5]:
cash_flow.movements

2020-01-01    100.0
2020-01-02    200.0
2019-01-01    300.0
2020-12-31   -100.0
Name: Operational Expenses, dtype: float64

In [6]:
cash_flow.movements.index

DatetimeIndex(['2020-01-01', '2020-01-02', '2019-01-01', '2020-12-31'], dtype='datetime64[ns]', freq=None)

In [7]:
cash_flow.units

<Unit('AUD')>

In [8]:
cash_flow.display()

Name: Operational Expenses
Units: AUD
Movements: 
|                date |   Operational Expenses |
|---------------------|------------------------|
| 2020-01-01 00:00:00 |                $100.00 |
| 2020-01-02 00:00:00 |                $200.00 |
| 2019-01-01 00:00:00 |                $300.00 |
| 2020-12-31 00:00:00 |               -$100.00 |




In [9]:
# Define a Span:
start_date = pd.Timestamp('2001-01-01')
num_periods = 11
span = rk.span.Span.from_num_periods(
    name='Operation',
    date=start_date,
    period_type=rk.periodicity.Type.YEAR,
    num_periods=num_periods)
print(span)

Span: Operation
Start Date: 2001-01-01
End Date: 2011-12-31


In [10]:
# Define a Compounding Projection:
compounding_rate = 0.02
projection = rk.projection.Extrapolation(
    form=rk.extrapolation.Compounding(rate=compounding_rate),
    sequence=span.to_index(period_type=rk.periodicity.Type.YEAR))

In [11]:
# Define a compounding Cash Flow:
initial_income = 100 * currency.units
potential_gross_income = rk.flux.Flow.from_projection(
    name='Potential Gross Income',
    value=initial_income,
    proj=projection,
    units=currency.units)
potential_gross_income

date,Potential Gross Income
2001-12-31 00:00:00,$100.00
2002-12-31 00:00:00,$102.00
2003-12-31 00:00:00,$104.04
2004-12-31 00:00:00,$106.12
2005-12-31 00:00:00,$108.24
2006-12-31 00:00:00,$110.41
2007-12-31 00:00:00,$112.62
2008-12-31 00:00:00,$114.87
2009-12-31 00:00:00,$117.17
2010-12-31 00:00:00,$119.51


In [12]:
vacancy_rate = 0.05
vacancy = rk.flux.Flow(
    name='Vacancy Allowance',
    movements=potential_gross_income.movements * -vacancy_rate,
    units=currency.units)
vacancy

date,Vacancy Allowance
2001-12-31 00:00:00,-$5.00
2002-12-31 00:00:00,-$5.10
2003-12-31 00:00:00,-$5.20
2004-12-31 00:00:00,-$5.31
2005-12-31 00:00:00,-$5.41
2006-12-31 00:00:00,-$5.52
2007-12-31 00:00:00,-$5.63
2008-12-31 00:00:00,-$5.74
2009-12-31 00:00:00,-$5.86
2010-12-31 00:00:00,-$5.98


In [13]:
effective_gross_income = rk.flux.Stream(
    name='Effective Gross Income',
    flows=[potential_gross_income, vacancy],
    period_type=rk.periodicity.Type.YEAR)
effective_gross_income

date,Potential Gross Income,Vacancy Allowance
2001,$100.00,-$5.00
2002,$102.00,-$5.10
2003,$104.04,-$5.20
2004,$106.12,-$5.31
2005,$108.24,-$5.41
2006,$110.41,-$5.52
2007,$112.62,-$5.63
2008,$114.87,-$5.74
2009,$117.17,-$5.86
2010,$119.51,-$5.98


In [14]:
effective_gross_income_flow = effective_gross_income.sum()
effective_gross_income_flow

date,Effective Gross Income (sum)
2001-12-31 00:00:00,$95.00
2002-12-31 00:00:00,$96.90
2003-12-31 00:00:00,$98.84
2004-12-31 00:00:00,$100.81
2005-12-31 00:00:00,$102.83
2006-12-31 00:00:00,$104.89
2007-12-31 00:00:00,$106.99
2008-12-31 00:00:00,$109.13
2009-12-31 00:00:00,$111.31
2010-12-31 00:00:00,$113.53


In [15]:
opex_pgi_ratio = .35
operating_expenses = rk.flux.Flow(
    name='Operating Expenses',
    movements=potential_gross_income.movements * opex_pgi_ratio,
    units=currency.units).invert()

net_operating_income = rk.flux.Stream(
    name='Net Operating Income',
    flows=[effective_gross_income_flow, operating_expenses],
    period_type=rk.periodicity.Type.YEAR)

net_operating_income

date,Effective Gross Income (sum),Operating Expenses
2001,$95.00,-$35.00
2002,$96.90,-$35.70
2003,$98.84,-$36.41
2004,$100.81,-$37.14
2005,$102.83,-$37.89
2006,$104.89,-$38.64
2007,$106.99,-$39.42
2008,$109.13,-$40.20
2009,$111.31,-$41.01
2010,$113.53,-$41.83


In [16]:
capex_pgi_ratio = .1
capital_expenditures = rk.flux.Flow(
    name='Capital Expenditures',
    movements=potential_gross_income.movements * capex_pgi_ratio,
    units=currency.units).invert()

net_annual_cashflows = rk.flux.Stream(
    name='Net Annual Cashflows',
    flows=[net_operating_income.sum(), capital_expenditures],
    period_type=rk.periodicity.Type.YEAR)

net_annual_cashflows

date,Net Operating Income (sum),Capital Expenditures
2001,$60.00,-$10.00
2002,$61.20,-$10.20
2003,$62.42,-$10.40
2004,$63.67,-$10.61
2005,$64.95,-$10.82
2006,$66.24,-$11.04
2007,$67.57,-$11.26
2008,$68.92,-$11.49
2009,$70.30,-$11.72
2010,$71.71,-$11.95


In [17]:
reversion_span = rk.span.Span.from_num_periods(
    name='Reversion',
    date=start_date + pd.DateOffset(years=9),
    period_type=rk.periodicity.Type.YEAR,
    num_periods=1)

exit_caprate = 0.05
reversion_flow = rk.flux.Flow.from_projection(
    name='Reversion',
    value=net_annual_cashflows.sum().movements.values[-1] / exit_caprate,
    proj=rk.projection.Distribution(
        form=rk.distribution.Uniform(),
        sequence=reversion_span.to_index(period_type=rk.periodicity.Type.YEAR)),
    units=currency.units)
reversion_flow

date,Reversion
2010-12-31 00:00:00,"$1,218.99"


In [18]:
net_cashflows_with_reversion = rk.flux.Stream(
    name='Net Cashflow with Reversion',
    flows=[net_annual_cashflows.sum(), reversion_flow],
    period_type=rk.periodicity.Type.YEAR).trim_to_span(
    rk.span.Span(
        start_date=start_date,
        end_date=reversion_span.end_date)
    )
net_cashflows_with_reversion

date,Net Annual Cashflows (sum),Reversion
2001,$50.00,0
2002,$51.00,0
2003,$52.02,0
2004,$53.06,0
2005,$54.12,0
2006,$55.20,0
2007,$56.31,0
2008,$57.43,0
2009,$58.58,0
2010,$59.75,"$1,218.99"


In [19]:
table = rk.flux.Stream(
    name='Table 1.1',
    flows=[
        potential_gross_income,
        vacancy,
        effective_gross_income_flow,
        operating_expenses,
        net_operating_income.sum(),
        capital_expenditures,
        net_annual_cashflows.sum(),
        reversion_flow,
        net_cashflows_with_reversion.sum()
        ],
    period_type=rk.periodicity.Type.YEAR
    )
table

date,Potential Gross Income,Vacancy Allowance,Effective Gross Income (sum),Operating Expenses,Net Operating Income (sum),Capital Expenditures,Net Annual Cashflows (sum),Reversion,Net Cashflow with Reversion (sum)
2001,$100.00,-$5.00,$95.00,-$35.00,$60.00,-$10.00,$50.00,0,$50.00
2002,$102.00,-$5.10,$96.90,-$35.70,$61.20,-$10.20,$51.00,0,$51.00
2003,$104.04,-$5.20,$98.84,-$36.41,$62.42,-$10.40,$52.02,0,$52.02
2004,$106.12,-$5.31,$100.81,-$37.14,$63.67,-$10.61,$53.06,0,$53.06
2005,$108.24,-$5.41,$102.83,-$37.89,$64.95,-$10.82,$54.12,0,$54.12
2006,$110.41,-$5.52,$104.89,-$38.64,$66.24,-$11.04,$55.20,0,$55.20
2007,$112.62,-$5.63,$106.99,-$39.42,$67.57,-$11.26,$56.31,0,$56.31
2008,$114.87,-$5.74,$109.13,-$40.20,$68.92,-$11.49,$57.43,0,$57.43
2009,$117.17,-$5.86,$111.31,-$41.01,$70.30,-$11.72,$58.58,0,$58.58
2010,$119.51,-$5.98,$113.53,-$41.83,$71.71,-$11.95,$59.75,"$1,218.99","$1,278.75"


In [20]:
discount_rate = 0.07
pvs = net_cashflows_with_reversion.sum().pv(
    name='Present Value',
    period_type=rk.periodicity.Type.YEAR,
    discount_rate=discount_rate)
pvs

date,Present Value
2001-12-31 00:00:00,$46.73
2002-12-31 00:00:00,$44.55
2003-12-31 00:00:00,$42.46
2004-12-31 00:00:00,$40.48
2005-12-31 00:00:00,$38.59
2006-12-31 00:00:00,$36.78
2007-12-31 00:00:00,$35.07
2008-12-31 00:00:00,$33.43
2009-12-31 00:00:00,$31.87
2010-12-31 00:00:00,$650.05


In [21]:
property_pv = pvs.collapse().movements.item()
print('Property PV: ${:,.0f}'.format(property_pv))

Property PV: $1,000
